In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
#r = requests.get(kite.login_url())
#redirect_link = r.url
#print(redirect_link)
#request_token = redirect_link[]

In [11]:
# Get the Request Token from the redirect URL

request_token = "n58YvWghweGwkdwMYcxYIXRks2mLMFLh"

# Get Access Token

In [12]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "1wXmq4ixlf6mrXCBoABlG4abPDrFIdrl"
kite.set_access_token(access_token)
print(access_token)

1wXmq4ixlf6mrXCBoABlG4abPDrFIdrl


In [13]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BFO', 'CDS', 'BSE', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [14]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [15]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [16]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [17]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [24]:
def place_trade(trade_log, current_time, current_price, current_smi, current_signal, current_transaction_type):
    print(current_time, current_transaction_type, current_price)
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_smi, current_signal, current_transaction_type, current_status]
    return trade_log, current_transaction_type, current_status

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [21]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20AUGFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=21, hour=15 , minute=30 , second=0)

# THE PAYLOAD

In [22]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 897509


In [28]:
trade_log = pd.DataFrame(columns=['time','price','smi','signal','transaction_type','status'])
current_status='closed'
while True:
    test_end = datetime.now()
    test_start = test_end - timedelta(days=1)
    hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
    smi_df = get_smi(hist_data, k_period, k_smooth_period, d_smooth_period, d_period)
    current_time = list(smi_df.index)[-1]
    current_price = smi_df.loc[current_time, 'close']
    current_smi = smi_df.loc[current_time, 'smi']
    current_signal = smi_df.loc[current_time, 'signal']
    if datetime.strftime(test_end, "%H")=='15' and int(datetime.strftime(test_end, "%M"))>=27:
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
                break
        else:
            break
    elif (current_smi-current_signal>5 and current_status=='closed') or (current_smi-current_signal>-8 and current_status=='open'):
        if current_status=='closed':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            sleep(180)
        elif current_status=='open' and current_transaction_type=='buy':
            continue
        elif current_status=='open' and current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            #trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            sleep(180)
    elif (current_smi-current_signal<-5 and current_status=='closed') or (current_smi-current_signal<8 and current_status=='open'):
        if current_status=='closed':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            sleep(180)
        elif current_status=='open' and current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            #trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_smi=current_smi, current_signal=current_signal, current_transaction_type=transaction_type)
            sleep(180)
        elif current_status=='open' and current_transaction_type=='sell':
            continue
    sleep(1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A11%3A08&to=2020-08-21+13%3A11%3A08&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2490
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A11%3A10&to=2020-08-21+13%3A11%3A10&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2490
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A11%3A12&to=2020-08-21+13%3A11%3A12&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2491
DEBUG:urllib3.connectionpool:Starting

2020-08-21 13:09:00 buy 22249.85


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A14%3A43&to=2020-08-21+13%3A14%3A43&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2495
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A14%3A43&to=2020-08-21+13%3A14%3A43&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2495
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A14%3A44&to=2020-08-21+13%3A14%3A44&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2495
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A14%3A59&to=2020-08-21+13%3A14%3A59&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2493
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A14%3A59&to=2020-08-21+13%3A14%3A59&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2493
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A15%3A00&to=2020-08-21+13%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2493
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A15%3A14&to=2020-08-21+13%3A15%3A14&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2515
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A15%3A15&to=2020-08-21+13%3A15%3A15&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2518
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A15%3A15&to=2020-08-21+13%3A15%3A15&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2518
DEBUG:urllib3.connectionpool:Starting

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A11&to=2020-08-21+13%3A22%3A11&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2504
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A12&to=2020-08-21+13%3A22%3A12&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2504
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A13&to=2020-08-21+13%3A22%3A13&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2505
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A28&to=2020-08-21+13%3A22%3A28&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2504
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A29&to=2020-08-21+13%3A22%3A29&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2504
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A30&to=2020-08-21+13%3A22%3A30&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2502
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A44&to=2020-08-21+13%3A22%3A44&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2507
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A45&to=2020-08-21+13%3A22%3A45&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2507
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A22%3A45&to=2020-08-21+13%3A22%3A45&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2506
DEBUG:urllib3.connectionpool:Starting

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A23%3A05&to=2020-08-21+13%3A23%3A05&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2493
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A23%3A06&to=2020-08-21+13%3A23%3A06&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2494
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/11348226/3minute?instrument_token=11348226&from=2020-08-20+13%3A23%3A08&to=2020-08-21+13%3A23%3A08&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2492
DEBUG:urllib3.connectionpool:Starting

ReadTimeout: HTTPSConnectionPool(host='api.kite.trade', port=443): Read timed out. (read timeout=7)

In [29]:
trade_log.to_excel('LIVE Deployment Logs\RUN1B - 3min_candle_SMI-Signal_gap_trigger.xlsx')